In [59]:
import SimpleITK as sitk
import os
import pandas as pd
import json
import numpy as np

In [60]:
# data path
input_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'
output_directory = 'C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng'

In [61]:
# Read / write SWC files
# To do - move this to a library
scols = ["Index","Type","X","Y","Z","Radius","Parent"]
sdict = {}
for idx in range(len(scols)) :
    sdict[idx] = scols[idx]

def read_swc (fname) :
    
    df = pd.read_csv( fname, sep=r"\s+", comment = "#", header=None )
    
    # keep only the 7 standard columns
    dcols = df.columns[len(scols):]
    df = df.drop( dcols, axis = 1 )
    df = df.rename( columns = sdict )
    df.set_index('Index', inplace=True)
    
    return df

def write_swc (df, fname) :
    df.to_csv(fname,sep = ' ', header=False)


In [62]:
# open combined and downsampled inventory files
combined_file = 'combined_inventory.csv'
reconstruction_file = 'reconstruction_inventory.csv'

In [63]:
samples = pd.read_csv(os.path.join(input_directory,combined_file))
reconstructions = pd.read_csv(os.path.join(input_directory,reconstruction_file))

samples.set_index('HUST Sample ID', inplace=True)
reconstructions.set_index('Reconstruction ID', inplace=True)

In [64]:
samples = samples.loc[[17543,182725]]

In [65]:
# mapping from rotation value in json to orientation
rotation = {}
rotation[0] = 'SLA'

axes = {}
axes[0] = 'X'
axes[1] = 'Y'
axes[2] = 'Z'

oext = ".swc"

for s, row in samples.iterrows() :
    
    print(s)
    
    dname = row['Sample ID']
    
    #
    # create output morphology directory
    #
    odir = os.path.join( output_directory, dname, 'morphology')
    if not os.path.exists( odir ) :
        os.makedirs( odir )
    
    #
    # open json metadata file
    #
    if pd.isna(row['JSON Files']) :
        continue
        
    jname = row['JSON Files'].split(';')[0]
    jpath = os.path.join( input_directory, dname, jname )
    
    with open(jpath,) as jf :
        jdata = json.load(jf)
        
    resolution = jdata['SampleInfo']['VoxelResolution '] # in micrometers
    
    otag = rotation[jdata['SampleInfo']['Rotation']]
    
    
    #
    # Find associated reconstruction files
    #
    rdf = reconstructions
    rdf = rdf[rdf["HUST Sample ID"] == s]
    rdf = rdf[pd.isna(rdf["Reconstruction Type"])]
    
    for swc, row in rdf.iterrows() :
        
        fpath = os.path.join( input_directory, "cells", dname, "1", row['Filename'] )
        print(" : " +  fpath)
        
        # open swc file with voxel index coordinates
        ndf = read_swc(fpath)
        
        # convert to microns
        for a in axes :
            ndf[axes[a]] = ndf[axes[a]] * resolution[a]
            
        # construct output file path
        oname = os.path.splitext(row['Filename'])[0] + '_um' + '_'  + otag + oext 
        opath = os.path.join( odir, oname ) 
        print( "  : " + opath )
        
        # write converted swc
        write_swc( ndf, opath )
        
        


17543
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\cells\mouseID_325875-17543\1\1059283974_17543_3828-X6023-Y17044.swc
  : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\mouseID_325875-17543\morphology\1059283974_17543_3828-X6023-Y17044_um_SLA.swc
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\cells\mouseID_325875-17543\1\1059284283_17543_3471-X12087-Y3637.swc
  : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\mouseID_325875-17543\morphology\1059284283_17543_3471-X12087-Y3637_um_SLA.swc
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\cells\mouseID_325875-17543\1\1059285168_17543_5334-X5066-Y20381.swc
  : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\mouseID_325875-17543\morphology\1059285168_17543_5334-X5066-Y20381_um_SLA.swc
 : C:/Users/lydia/OneDrive/Work/DigitalAssets/BICCNMorphologyInventory/u19zeng\